In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset('conll2003')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
label_names = data["train"].features['ner_tags'].feature.names

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
idx= 0
t = tokenizer(data['train'][idx]["tokens"],is_split_into_words=True)
t

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
begin2inside = {
    1:2,
    3:4,
    5:6,
    7:8,
}

In [ ]:
def align_targets(labels, word_ids):
  aligned_labels = []
  last_word = None
  for word in word_ids:
    if word is None:
      label = -100
    elif word != last_word:
      label = labels[word]
    else:
      label = labels[word]
      if label in begin2inside:
        label = begin2inside[label]

    aligned_labels.append(label)

    last_word = word

  return aligned_labels

In [ ]:
labels = data['train'][idx]['ner_tags']
word_ids = t.word_ids()
aligned_targets = align_targets(labels, word_ids)
aligned_targets

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [ ]:
aligned_labels = [label_names[t] if t>=0 else None for t in aligned_targets]
for x,y in zip(t.tokens(), aligned_labels):
  print(f"{x}\t{y}")

[CLS]	None
EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
la	O
##mb	O
.	O
[SEP]	None


In [ ]:
words = [
  '[CLS]', 'Ger', '##man', 'call', 'to', 'boycott', 'Micro', '##soft', '[SEP]']
word_ids = [None, 0, 0, 1, 2, 3, 4, 4, None]
labels = [7, 0, 0, 0, 3]
aligned_targets = align_targets(labels, word_ids)
aligned_labels = [label_names[t] if t >= 0 else None for t in aligned_targets]
for x, y in zip(words, aligned_labels):
  print(f"{x}\t{y}")

[CLS]	None
Ger	B-MISC
##man	I-MISC
call	O
to	O
boycott	O
Micro	B-ORG
##soft	I-ORG
[SEP]	None


In [ ]:
def tokenize_fn(batch):
  tokenized_inputs = tokenizer(
    batch['tokens'], truncation=True, is_split_into_words=True
  )

  labels_batch = batch['ner_tags']
  aligned_labels_batch = []
  for i, labels in enumerate(labels_batch):
    word_ids = tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_targets(labels, word_ids))

  tokenized_inputs['labels'] = aligned_labels_batch

  return tokenized_inputs

In [ ]:
tokenized_datasets = data.map(
    tokenize_fn,
    batched=True,
    remove_columns = data['train'].column_names
)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch['labels']

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric('seqeval')

<ipython-input-21-1d148ab27cb7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('seqeval')


In [ ]:
metric.compute(
    predictions = [[0,0,0]],
    references=[[0,0,1]]
)

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [ ]:
import numpy as np

In [ ]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  preds = np.argmax(logits, axis=-1)
  str_labels = [
    [label_names[t] for t in label if t != -100] for label in labels
  ]

  str_preds = [
    [label_names[p] for p, t in zip(pred, targ) if t != -100] \
      for pred, targ in zip(preds, labels)
  ]

  the_metrics = metric.compute(predictions=str_preds, references=str_labels)
  return {
    'precision': the_metrics['overall_precision'],
    'recall': the_metrics['overall_recall'],
    'f1': the_metrics['overall_f1'],
    'accuracy': the_metrics['overall_accuracy'],
  }

In [ ]:
id2label = {k: v for k, v in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

In [ ]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
training_args = TrainingArguments(
    "distilbert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args= training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.103200,0.081820,0.895717,0.915012,0.905261,0.976497
2,0.046000,0.074275,0.904262,0.928307,0.916127,0.981221
3,0.026600,0.067983,0.914000,0.937227,0.925467,0.983414


TrainOutput(global_step=5268, training_loss=0.08048465293559869, metrics={'train_runtime': 331.0681, 'train_samples_per_second': 127.234, 'train_steps_per_second': 15.912, 'total_flos': 462023079274890.0, 'train_loss': 0.08048465293559869, 'epoch': 3.0})

In [ ]:
trainer.save_model('my_saved_model')
from transformers import pipeline

In [ ]:
ner= pipeline(
    'token-classification',
    model='my_saved_model',
    aggregation_strategy='simple',
    device=0

)

In [ ]:
s = 'Bill Gates was the CEO of Microsoft in Seattle, Washington.'
ner(s)

[{'entity_group': 'PER',
  'score': 0.9995009,
  'word': 'Bill Gates',
  'start': 0,
  'end': 10},
 {'entity_group': 'ORG',
  'score': 0.99897385,
  'word': 'Microsoft',
  'start': 26,
  'end': 35},
 {'entity_group': 'LOC',
  'score': 0.999433,
  'word': 'Seattle',
  'start': 39,
  'end': 46},
 {'entity_group': 'LOC',
  'score': 0.99929225,
  'word': 'Washington',
  'start': 48,
  'end': 58}]

In [ ]:
import nltk
from nltk.corpus import brown

In [ ]:
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
corpus = brown.tagged_sents(tagset='univeral')
corpus

[[('The', 'UNK'), ('Fulton', 'UNK'), ('County', 'UNK'), ('Grand', 'UNK'), ('Jury', 'UNK'), ('said', 'UNK'), ('Friday', 'UNK'), ('an', 'UNK'), ('investigation', 'UNK'), ('of', 'UNK'), ("Atlanta's", 'UNK'), ('recent', 'UNK'), ('primary', 'UNK'), ('election', 'UNK'), ('produced', 'UNK'), ('``', 'UNK'), ('no', 'UNK'), ('evidence', 'UNK'), ("''", 'UNK'), ('that', 'UNK'), ('any', 'UNK'), ('irregularities', 'UNK'), ('took', 'UNK'), ('place', 'UNK'), ('.', 'UNK')], [('The', 'UNK'), ('jury', 'UNK'), ('further', 'UNK'), ('said', 'UNK'), ('in', 'UNK'), ('term-end', 'UNK'), ('presentments', 'UNK'), ('that', 'UNK'), ('the', 'UNK'), ('City', 'UNK'), ('Executive', 'UNK'), ('Committee', 'UNK'), (',', 'UNK'), ('which', 'UNK'), ('had', 'UNK'), ('over-all', 'UNK'), ('charge', 'UNK'), ('of', 'UNK'), ('the', 'UNK'), ('election', 'UNK'), (',', 'UNK'), ('``', 'UNK'), ('deserves', 'UNK'), ('the', 'UNK'), ('praise', 'UNK'), ('and', 'UNK'), ('thanks', 'UNK'), ('of', 'UNK'), ('the', 'UNK'), ('City', 'UNK'), ('of